# Hurricane Fiona (2022-09-16/22) around Puerto Rico


## Download GOES-16

In [ ]:
from pathlib import Path

path = Path("data", "goes16-hurricane")
path.mkdir(parents=True, exist_ok=True)

In [ ]:
min_lon, max_lon, min_lat, max_lat = bbox = -72, -61, 15, 21

min_time = "2022-09-16T12:00:00Z"
max_time = "2022-09-22T00:00:00Z"

In [ ]:
import folium
import shapely

aoi = shapely.box(xmin=min_lon, ymin=min_lat, xmax=max_lon, ymax=max_lat)

m = folium.Map()
folium.GeoJson(aoi).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
from datetime import datetime

from goes2go import GOES

blend = "ABI-L2-MCMIPF"

start = datetime.strptime(min_time, "%Y-%m-%dT%H:%M:%SZ")
end = datetime.strptime(max_time, "%Y-%m-%dT%H:%M:%SZ")

g = GOES(satellite=16, product=blend, domain="C")
df = g.df(
    start=start,
    end=end,
)

df

# Do not run this cell unless you have a few days and ~256 G of space to spare!!!

For this particular time-rage it will break down to:

|size | day |
|-----|-----|
| 26G | 259 |
| 46G |	260 |
| 46G |	261 |
| 46G |	262 |
| 46G |	263 |
| 46G |	264 |

In [ ]:
g.timerange(start=start, end=end, save_dir=path)

## Save True/NaturalColor slices

In [ ]:
import gc

# This is bad but I could not find a better single file loader in goes2go
from goes2go.data import _as_xarray_MP

from utils import smaller_image


def save_smaller_nc(k):
    fname = df.iloc[k].file.split("/")[-1]
    fname = f"rgb-{fname}"
    fname = path.joinpath(fname)
    if not fname.exists():
        print(f"Saving {fname}")
        ds = _as_xarray_MP(df.iloc[k].file, save_dir=path, i=1, n=1, verbose=True)
        rgb, imshow_kw = smaller_image(ds, min_lon, max_lon, min_lat, max_lat)
        rgb.to_netcdf(fname)
        # Closing, deleting, and calling the garbage collector
        # is only required when dealing with many files.
        # This avoids running out of memory just b/c we have may `ds`s laying around.
        rgb.close()
        ds.close()
        del rgb
        del ds
        gc.collect()
    else:
        print(f"{fname} already exist")


for k in range(len(df)):
    save_smaller_nc(k)

# NB: One can delete all the files but the first one when this is done. We need one full file to extract the projection info later. I could fix this by storing that info too but I'm kind of lazy.